## 공통 코드

In [27]:
import sys
# sklearn ≥0.20 필수
import sklearn
# 공통 모듈 임포트
import numpy as np
import pandas as pd
import os
# 노트북 실행 결과를 동일하게 유지하기 위해
np.random.seed(42)
# 깔끔한 그래프 출력을 위해
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

import platform
from matplotlib import font_manager, rc
#매킨토시의 경우
if platform.system() == 'Darwin':
    rc('font', family='AppleGothic')
#윈도우의 경우
elif platform.system() == 'Windows':
    font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
    rc('font', family=font_name)
mpl.rcParams['axes.unicode_minus'] = False

# 그림을 저장할 위치
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "natural_language_processing"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)
def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("그림 저장:", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)
import warnings
warnings.filterwarnings(action='ignore')

### 문서 군집

### 디렉토리 내의 .datafh 끝나는 파일을 모두 읽기

In [16]:
import glob ,os
# 디렉토리 이름을 생성
path = '.\\data\\OpinosisDataset1.0\\topics' 
# 디렉토리 안의 모든 파일 이름을 list 로 생성
all_files = glob.glob(os.path.join(path, "*.data"))
print(all_files)

['.\\data\\OpinosisDataset1.0\\topics\\accuracy_garmin_nuvi_255W_gps.txt.data', '.\\data\\OpinosisDataset1.0\\topics\\bathroom_bestwestern_hotel_sfo.txt.data', '.\\data\\OpinosisDataset1.0\\topics\\battery-life_amazon_kindle.txt.data', '.\\data\\OpinosisDataset1.0\\topics\\battery-life_ipod_nano_8gb.txt.data', '.\\data\\OpinosisDataset1.0\\topics\\battery-life_netbook_1005ha.txt.data', '.\\data\\OpinosisDataset1.0\\topics\\buttons_amazon_kindle.txt.data', '.\\data\\OpinosisDataset1.0\\topics\\comfort_honda_accord_2008.txt.data', '.\\data\\OpinosisDataset1.0\\topics\\comfort_toyota_camry_2007.txt.data', '.\\data\\OpinosisDataset1.0\\topics\\directions_garmin_nuvi_255W_gps.txt.data', '.\\data\\OpinosisDataset1.0\\topics\\display_garmin_nuvi_255W_gps.txt.data', '.\\data\\OpinosisDataset1.0\\topics\\eyesight-issues_amazon_kindle.txt.data', '.\\data\\OpinosisDataset1.0\\topics\\features_windows7.txt.data', '.\\data\\OpinosisDataset1.0\\topics\\fonts_amazon_kindle.txt.data', '.\\data\\Opinos

In [17]:
# 파일의 이름을 저장할 내용~ ㄸㄱㄹㅇㄹ
filename_list = []
opinion_text = []

for file_ in all_files:
    # 개별 파일을 읽어서 DataFrame으로 생성
    df = pd.read_table(file_, index_col=None, header=0, encoding='latin1')
    # 절대경로로 주어진 file 명을 가공. 
    # 만일 Windows에서 수행시에는 아래 /를 \\ 변경. 맨 마지막 .data 확장자도 제거
    filename_ = file_.split('\\')[-1]
    filename = filename_.split('.')[0]
    #파일명 리스트와 파일내용 리스트에 파일명과 파일 내용을 추가.
    filename_list.append(filename)
    opinion_text.append(df.to_string())

In [20]:
# print(filename_list)
# print(opinion_text[0])

# 파일 이름과 내용으로 DataFrame 생성
document_df = pd.DataFrame({'filename':filename_list, 'opinion_text':opinion_text})
document_df.head()

,filename,opinion_text
0,accuracy_garmin_nuvi_255W_gps,...
1,bathroom_bestwestern_hotel_sfo,...
2,battery-life_amazon_kindle,...
3,battery-life_ipod_nano_8gb,...
4,battery-life_netbook_1005ha,...


### 피처 벡터화

In [32]:
from nltk.stem import WordNetLemmatizer
import nltk
import string

# 구두점 제거
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
# print(remove_punct_dict)
lemmar = WordNetLemmatizer()
def LemTokens(tokens):
    return [lemmar.lemmatize(token) for token in tokens]

def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vect = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english',
                             ngram_range=(1,2), min_df=0.05, max_df=0.85)

# 문자에 어떤 단어가 얼마의 가중치를 가지고 있는지 희소 행렬로 생성
feature_vect = tfidf_vect.fit_transform(document_df['opinion_text'])
print(feature_vect)

  (0, 1465)	0.023283541665665836
  (0, 385)	0.023283541665665836
  (0, 1470)	0.05108766228354363
  (0, 2652)	0.023283541665665836
  (0, 1357)	0.02182613746339447
  (0, 723)	0.023283541665665836
  (0, 4367)	0.02182613746339447
  (0, 4593)	0.01798716138533979
  (0, 4273)	0.02063535152550281
  (0, 1468)	0.019628556360506585
  (0, 1804)	0.019628556360506585
  (0, 4040)	0.023283541665665836
  (0, 1467)	0.015101445414252302
  (0, 2469)	0.023283541665665836
  (0, 2173)	0.02182613746339447
  (0, 4191)	0.02182613746339447
  (0, 4196)	0.019628556360506585
  (0, 234)	0.023283541665665836
  (0, 4041)	0.023283541665665836
  (0, 4092)	0.04127070305100562
  (0, 1533)	0.04365227492678894
  (0, 1996)	0.023283541665665836
  (0, 158)	0.02182613746339447
  (0, 1164)	0.023283541665665836
  (0, 3130)	0.02182613746339447
  :	:
  (50, 2028)	0.01659453509406821
  (50, 2037)	0.009111906180203912
  (50, 1655)	0.009761754136944606
  (50, 1321)	0.009452031092583052
  (50, 3294)	0.022126046792090945
  (50, 3430)	0.

### 군집 알고리즘 수행

In [33]:
from sklearn.cluster import KMeans

# 5개의 군집을 위한 인스턴스 생성
km_cluster = KMeans(n_clusters=5, max_iter=10000, random_state=0)
# 훈련
km_cluster.fit(feature_vect)
# 각 데이터의 레이블을 저장
cluster_label = km_cluster.labels_
# 각 군집의 중앙점을 저장
cluster_centers = km_cluster.cluster_centers_

document_df['cluster_label'] = cluster_label

# document_df.head()

In [34]:
document_df[document_df['cluster_label']==1].sort_values(by='filename')

,filename,opinion_text,cluster_label
2,battery-life_amazon_kindle,...,1
3,battery-life_ipod_nano_8gb,...,1
4,battery-life_netbook_1005ha,...,1
19,keyboard_netbook_1005ha,...,1
26,performance_netbook_1005ha,...,1
41,size_asus_netbook_1005ha,...,1
42,sound_ipod_nano_8gb,headphone jack i got a clear case for it a...,1
44,speed_windows7,...,1


In [35]:
from sklearn.cluster import KMeans
# 3개의 집합으로 군집화
km_cluster = KMeans(n_clusters=3, max_iter=10000, random_state=0)
km_cluster.fit(feature_vect)
cluster_label = km_cluster.labels_
# 소속 클러스터를 cluster_label 컬럼으로 할당하고 cluster_label 값으로 정렬
document_df['cluster_label'] = cluster_label
document_df.sort_values(by='cluster_label')

,filename,opinion_text,cluster_label
0,accuracy_garmin_nuvi_255W_gps,...,0
48,updates_garmin_nuvi_255W_gps,...,0
44,speed_windows7,...,0
43,speed_garmin_nuvi_255W_gps,...,0
42,sound_ipod_nano_8gb,headphone jack i got a clear case for it a...,0
41,size_asus_netbook_1005ha,...,0
36,screen_netbook_1005ha,...,0
35,screen_ipod_nano_8gb,...,0
34,screen_garmin_nuvi_255W_gps,...,0
33,satellite_garmin_nuvi_255W_gps,...,0


### 군집을 이루게 만든 핵심 단어 출력

In [36]:
# 클러스터링 모델, 데이터, 피처 이름, 클러스터 개수, 추출한 핵심 단어 개수를 받아서
# 리턴하는 함수
def get_cluster_details(cluster_model, cluster_data, feature_names, clusters_num, top_n_features=10):
    cluster_details = {}

    # 군집 중심과의 거리가 먼 단어 순으로 저장하기
    centroid_feature_ordered_ind = cluster_model.cluster_centers_.argsort()[:,::-1]

    for cluster_num in range(clusters_num):
        # 개별 군집별 정보를 담을 데이터 초기화.
        cluster_details[cluster_num] = {}
        cluster_details[cluster_num]['cluster'] = cluster_num
        # 중요 피처를 추출
        top_feature_indexes = centroid_feature_ordered_ind[cluster_num, :top_n_features]
        top_features = [feature_names[ind] for ind in top_feature_indexes ]
        # 중앙점과의 거리 저장
        top_feature_values = cluster_model.cluster_centers_[
            cluster_num, top_feature_indexes].tolist() 

        cluster_details[cluster_num]['top_features'] = top_features
        cluster_details[cluster_num]['top_features_value'] = top_feature_values
        filenames = cluster_data[cluster_data['cluster_label'] == cluster_num]['filename']
        filenames = filenames.values.tolist()
        cluster_details[cluster_num]['filenames'] = filenames
    return cluster_details

In [37]:
# 클러스터 별 핵심 단어를 출력하는 함수
def print_cluster_details(cluster_details):
    for cluster_num, cluster_detail in cluster_details.items():
        print('##### Cluster {0} #####'.format(cluster_num))
        print('핵심 단어:', cluster_detail['top_features'])
        print('파일명 :',cluster_detail['filenames'])


In [40]:
# 피처 이름을 전부 가져오기
feature_names = tfidf_vect.get_feature_names_out()
# print(feature_names)

cluster_details = get_cluster_details(cluster_model=km_cluster, cluster_data=document_df,
                                      feature_names=feature_names, clusters_num=3, top_n_features=10)
print_cluster_details(cluster_details)

##### Cluster 0 #####
핵심 단어: ['screen', 'battery', 'keyboard', 'battery life', 'life', 'kindle', 'direction', 'video', 'size', 'voice']
파일명 : ['accuracy_garmin_nuvi_255W_gps', 'battery-life_amazon_kindle', 'battery-life_ipod_nano_8gb', 'battery-life_netbook_1005ha', 'buttons_amazon_kindle', 'directions_garmin_nuvi_255W_gps', 'display_garmin_nuvi_255W_gps', 'eyesight-issues_amazon_kindle', 'features_windows7', 'fonts_amazon_kindle', 'keyboard_netbook_1005ha', 'navigation_amazon_kindle', 'performance_netbook_1005ha', 'price_amazon_kindle', 'satellite_garmin_nuvi_255W_gps', 'screen_garmin_nuvi_255W_gps', 'screen_ipod_nano_8gb', 'screen_netbook_1005ha', 'size_asus_netbook_1005ha', 'sound_ipod_nano_8gb', 'speed_garmin_nuvi_255W_gps', 'speed_windows7', 'updates_garmin_nuvi_255W_gps', 'video_ipod_nano_8gb', 'voice_garmin_nuvi_255W_gps']
##### Cluster 1 #####
핵심 단어: ['interior', 'seat', 'mileage', 'comfortable', 'gas', 'gas mileage', 'transmission', 'car', 'performance', 'quality']
파일명 : ['com

## 문장 유사도 측정

### 코사인 유사도를 구하는 알고리즘

In [54]:
def cos_similarity(v1, v2):
    dot_product = np.dot(v1, v2)
    l2_norm = (np.sqrt(sum(np.square(v1))) * np.sqrt(sum(np.square(v2))))
    similarity = dot_product / l2_norm
    return similarity

In [51]:
# 여러 문서에 있는 단어들은 가중치를 낮춤
doc_list = ['if you take the blue pill, the story ends',
            'if you take the red pill, you stay in Wonderland',
            'if you take the red pill, I show you how deep the rabbit hole goes']

tfidf_vect_simple = TfidfVectorizer()
feature_vect_simple = tfidf_vect_simple.fit_transform(doc_list)

# 현재 결과는 희소 행렬이라 거리 계산을 못함
print(feature_vect_simple[0],'\n\n',feature_vect_simple[1])

feature_vect_dense = feature_vect_simple.todense()
print(feature_vect_dense[0],'\n\n',feature_vect_dense[1])

  (0, 2)	0.41556360057939173
  (0, 13)	0.41556360057939173
  (0, 8)	0.24543855687841593
  (0, 0)	0.41556360057939173
  (0, 15)	0.49087711375683185
  (0, 14)	0.24543855687841593
  (0, 17)	0.24543855687841593
  (0, 6)	0.24543855687841593 

   (0, 16)	0.39624495215024286
  (0, 7)	0.39624495215024286
  (0, 12)	0.39624495215024286
  (0, 10)	0.3013544995034864
  (0, 8)	0.2340286519091622
  (0, 15)	0.2340286519091622
  (0, 14)	0.2340286519091622
  (0, 17)	0.4680573038183244
  (0, 6)	0.2340286519091622
[[0.4155636  0.         0.4155636  0.         0.         0.
  0.24543856 0.         0.24543856 0.         0.         0.
  0.         0.4155636  0.24543856 0.49087711 0.         0.24543856]] 

 [[0.         0.         0.         0.         0.         0.
  0.23402865 0.39624495 0.23402865 0.         0.3013545  0.
  0.39624495 0.         0.23402865 0.23402865 0.39624495 0.4680573 ]]


In [56]:
# 거리 계산
vect1 = np.array(feature_vect_dense[0]).reshape(-1,)
vect2 = np.array(feature_vect_dense[1]).reshape(-1,)
vect3 = np.array(feature_vect_dense[2]).reshape(-1,)

#첫번째 문장과 두번째 문장의 feature vector로 두개 문장의 Cosine 유사도 추출
similarity_simple = cos_similarity(vect1, vect2 )
print('문장 1, 문장 2 Cosine 유사도: {0:.3f}'.format(similarity_simple))
similarity_simple = cos_similarity(vect1, vect3 )
print('문장 1, 문장 3 Cosine 유사도: {0:.3f}'.format(similarity_simple))
similarity_simple = cos_similarity(vect2, vect3 )
print('문장 2, 문장 3 Cosine 유사도: {0:.3f}'.format(similarity_simple))



문장 1, 문장 2 Cosine 유사도: 0.402
문장 1, 문장 3 Cosine 유사도: 0.404
문장 2, 문장 3 Cosine 유사도: 0.456


### API 를 확용한 코사인 유사도 구하기

In [57]:
from sklearn.metrics.pairwise import cosine_similarity
similarity_simple_pair = cosine_similarity(feature_vect_simple[0] , feature_vect_simple)
print(similarity_simple_pair)

[[1.         0.40207758 0.40425045]]


### 문서 군집의 코사인 유사도 확인

In [58]:
km_cluster = KMeans(n_clusters=3, max_iter=10000, random_state=0)
km_cluster.fit(feature_vect)
cluster_label = km_cluster.labels_
# 각 군집의 centroid
cluster_centers = km_cluster.cluster_centers_
document_df['cluster_label'] = cluster_label

In [60]:
# 1번 클러스터의 문서들 간의 코사인 유사도 확인
hotel_indexes = document_df[document_df['cluster_label']==1].index
print('1번 클러스터링 된 문서들의 DataFrame Index:', hotel_indexes)

similarity_pair = cosine_similarity(feature_vect[hotel_indexes[0]], feature_vect[hotel_indexes])
print(similarity_pair)

호텔로 클러스터링 된 문서들의 DataFrame Index: Int64Index([6, 7, 16, 17, 18, 22, 25, 29, 37, 47], dtype='int64')
[[1.         0.83969704 0.15655631 0.33044002 0.25981841 0.16544257
  0.27569738 0.18050974 0.65502034 0.06229873]]


### 한글 문서의 유사도 측정

In [61]:
from sklearn.feature_extraction.text import CountVectorizer
from konlpy.tag import Twitter
twitter = Twitter()
vectorizer = CountVectorizer(min_df = 1)
contents = ['우리 과일 먹으로 가자', '나는 고기를 좋아합니다',
            '나는 공원에서 산책하는 것을 싫어합니다', '안녕하세요 반갑습니다 그동안 잘 계셨어요']
# 한글 토큰화
contents_tokens = [twitter.morphs(row) for row in contents]
print(contents_tokens)

[['우리', '과일', '먹으로', '가자'], ['나', '는', '고기', '를', '좋아합니다'], ['나', '는', '공원', '에서', '산책', '하는', '것', '을', '싫어합니다'], ['안녕하세요', '반갑습니다', '그동안', '잘', '계셨어요']]


In [62]:
    # 토큰화 된 결과를 가지고 다시 문장을 생성 - 피처 벡터화 때문
    contents_for_vectorize = []
    for content in contents_tokens:
        sentence = ''
        for word in content:
            sentence = sentence + ' ' + word
        contents_for_vectorize.append(sentence)
        
    print(contents_for_vectorize)

[' 우리 과일 먹으로 가자', ' 나 는 고기 를 좋아합니다', ' 나 는 공원 에서 산책 하는 것 을 싫어합니다', ' 안녕하세요 반갑습니다 그동안 잘 계셨어요']
